In [66]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
import math
import warnings
from sklearn import linear_model

from IPython.display import display

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors

from sklearn import ensemble
from sklearn.model_selection import cross_val_score

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif

from timeit import default_timer as timer


In [3]:
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
#y2015.head(5)
#y2015.describe
#list(X)

In [6]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [7]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [9]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_Nov-2017,settlement_date_Nov-2018,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Oct-2018,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017,settlement_date_Sep-2018
0,nan,nan,23850.000,23850.000,23850.000,17.270,596.210,68046.000,24.710,1.000,...,0,0,0,0,0,0,0,0,0,0
1,nan,nan,15000.000,15000.000,15000.000,10.780,324.500,65000.000,29.280,0.000,...,0,0,0,0,0,0,0,0,0,0
2,nan,nan,7200.000,7200.000,7200.000,17.270,257.670,20000.000,13.210,1.000,...,1,0,0,0,0,0,0,0,0,0
3,nan,nan,12000.000,12000.000,12000.000,9.170,382.550,39400.000,26.320,0.000,...,0,0,0,0,0,0,0,0,0,0
4,nan,nan,11950.000,11950.000,11950.000,13.440,405.180,34000.000,10.200,0.000,...,0,0,0,0,0,0,0,0,0,0
5,nan,nan,20000.000,20000.000,20000.000,14.850,691.840,110000.000,12.450,0.000,...,0,0,0,0,0,0,0,0,0,0
6,nan,nan,24700.000,24700.000,24700.000,11.990,820.280,65000.000,16.060,1.000,...,0,0,0,0,0,0,0,0,0,0
7,nan,nan,17475.000,17475.000,17475.000,13.990,597.170,39750.000,23.580,0.000,...,0,0,0,0,0,0,0,0,0,0
8,nan,nan,18000.000,18000.000,18000.000,7.490,559.830,75000.000,25.650,0.000,...,0,0,0,0,0,0,0,0,0,0
9,nan,nan,16000.000,16000.000,16000.000,12.880,363.070,90000.000,6.390,0.000,...,0,0,0,0,0,0,0,0,0,0


In [54]:

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)



In [55]:
X = y2015.drop('loan_status', 1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)
X.drop('last_pymnt_amnt', axis=1, inplace=True)
X.drop('out_prncp', axis=1, inplace=True)
X.drop('out_prncp_inv', axis=1, inplace=True)
#


,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_Apr-2015,issue_d_Aug-2015,issue_d_Dec-2015,issue_d_Feb-2015,issue_d_Jan-2015,issue_d_Jul-2015,issue_d_Jun-2015,issue_d_Mar-2015,issue_d_May-2015,issue_d_Nov-2015,issue_d_Oct-2015,issue_d_Sep-2015,pymnt_plan_n,pymnt_plan_y,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,title_Business,title_Car financing,title_Credit Card/Auto Repair,title_Credit card refinancing,title_Debt consolidation,title_DebtC,title_Green loan,title_Home buying,title_Home improvement,title_Learning and training,title_Major purchase,title_Medical expenses,title_Moving and relocation,title_New Baby and New House (CC Consolidate),title_Other,title_Pay off Lowes Card,title_Paying off higher interest cards & auto,title_Prescription Drug and Medical Costs,title_SAVE,title_Simple Loan Until Contract Is Completed,title_Student Loan,title_Trying to come back to reality!,title_Vacation,title_considerate,title_new day,title_new kitchen for momma!,title_odymeds,initial_list_status_f,initial_list_status_w,last_pymnt_d_Apr-2015,last_pymnt_d_Apr-2016,last_pymnt_d_Apr-2017,last_pymnt_d_Apr-2018,last_pymnt_d_Aug-2015,last_pymnt_d_Aug-2016,last_pymnt_d_Aug-2017,last_pymnt_d_Aug-2018,last_pymnt_d_Dec-2015,last_pymnt_d_Dec-2016,last_pymnt_d_Dec-2017,last_pymnt_d_Dec-2018,last_pymnt_d_Feb-2015,last_pymnt_d_Feb-2016,last_pymnt_d_Feb-2017,last_pymnt_d_Feb-2018,last_pymnt_d_Jan-2015,last_pymnt_d_Jan-2016,last_pymnt_d_Jan-2017,last_pymnt_d_Jan-2018,last_pymnt_d_Jul-2015,last_pymnt_d_Jul-2016,last_pymnt_d_Jul-2017,last_pymnt_d_Jul-2018,last_pymnt_d_Jun-2015,last_pymnt_d_Jun-2016,last_pymnt_d_Jun-2017,last_pymnt_d_Jun-2018,last_pymnt_d_Mar-2015,last_pymnt_d_Mar-2016,last_pymnt_d_Mar-2017,last_pymnt_d_Mar-2018,last_pymnt_d_May-2015,last_pymnt_d_May-2016,last_pymnt_d_May-2017,last_pymnt_d_May-2018,last_pymnt_d_Nov-2015,last_pymnt_d_Nov-2016,last_pymnt_d_Nov-2017,last_pymnt_d_Nov-2018,last_pymnt_d_Oct-2015,last_pymnt_d_Oct-2016,last_pymnt_d_Oct-2017,last_pymnt_d_Oct-2018,last_pymnt_d_Sep-2015,last_pymnt_d_Sep-2016,last_pymnt_d_Sep-2017,last_pymnt_d_Sep-2018,next_pymnt_d_Aug-2018,next_pymnt_d_Dec-2018,next_pymnt_d_Feb-2018,next_pymnt_d_Feb-2019,next_pymnt_d_Jan-2019,next_pymnt_d_Sep-2018,last_credit_pull_d_Apr-2015,last_credit_pull_d_Apr-2016,last_credit_pull_d_Apr-2017,last_credit_pull_d_Apr-2018,last_credit_pull_d_Aug-2015,last_credit_pull_d_Aug-2016,last_credit_pull_d_Aug-2017,last_credit_pull_d_Aug-2018,last_credit_pull_d_Dec-2014,last_credit_pull_d_Dec-2015,last_credit_pull_d_Dec-2016,last_credit_pull_d_Dec-2017,last_credi

In [ ]:
X_new_20 = SelectKBest(f_classif, k=20).fit_transform(X, Y)
X_new_20.shape

In [74]:
X_new_10 = SelectKBest(f_classif, k=10).fit_transform(X, Y)
X_new_10.shape

/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 20 332] are constant.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


(421095, 10)

In [70]:
start_base = timer()
print(cross_val_score(rfc, X, Y, cv=10))
end_base = timer()
print(end_base - start_base)

/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 20 332] are constant.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


(421095, 20)

In [77]:
start20 = timer()
print(cross_val_score(rfc, X_new_20, Y, cv=10))
end20 = timer()
print(end20 - start20)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

[0.98708207 0.98860182 0.98883929 0.99081    0.98900499 0.98940869
 0.99121286 0.98981167 0.99012041 0.99016791]
108.97768390800047


In [78]:
start10 = timer()
print(cross_val_score(rfc, X_new_10, Y, cv=10))
end10 = timer()
print(end10 - start10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

[0.9743541  0.9740454  0.97304806 0.97698939 0.97349798 0.97347423
 0.97679721 0.97221365 0.97408982 0.97406607]
88.02827108199926


By using SelectKBest algorithm, even after removing the payment amount and outstanding principal data points the model was able to achieve a 97+% success rate with 10 features, resulting in an 88 second time, compared to the 98+% result and 265 second time of the base dataset.  